In [ ]:
# Importing the Keras libraries and packages
#import pretrainded models
from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.applications.vgg19 import VGG19
#import packages
from keras.models import Model
from keras.preprocessing import image
from tensorflow.keras.layers import Input, Lambda ,Dense ,Flatten ,Dropout
import numpy as np
import tensorflow as tf
import pandas as pd
import keras
import matplotlib.pyplot as plt
%matplotlib inline
import os
import cv2
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

In [ ]:
def load_image_data(directory):
    
    images = []
    labels = []
    for label in sorted(os.listdir(directory)):
        for image in os.listdir(directory + "/" + label ):
            img = cv2.resize(cv2.imread(directory + "/" + label+ "/" +image), (300, 300))
            images.append(img)
            labels.append(label)
    images = np.array(images)
    labels = np.array(labels)
    return (images,labels)

In [ ]:
x_test,y_test=load_image_data('vgg/test')

In [ ]:
len(x_test)

In [ ]:
x_test=x_test.astype(float)/255.0

In [ ]:
x_test1= np.expand_dims(x_test, axis = 1)

In [ ]:
x_test1[0].shape

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_test_cat = le.fit_transform(y_test)
y_test_cat = to_categorical(y_test_cat)
y_test_cat[0]

In [ ]:
model= keras.models.load_model("model.h5")

In [ ]:
import cv2
from cvzone.HandTrackingModule import HandDetector
import numpy as np
import math
import time
import os

In [ ]:
labels=list(np.unique(y_test))

In [ ]:
cap = cv2.VideoCapture(0)
detector = HandDetector(maxHands=1)

 
offset = 20
imgSize = 300
  
#labels = ["Hello", "Thumbs up"]
 
while True:
    success, img = cap.read()
    imgOutput = img.copy()
    hands, img = detector.findHands(img)
    if hands:
        hand = hands[0]
        x, y, w, h = hand['bbox']
 
        imgWhite = np.ones((imgSize, imgSize, 3), np.uint8) * 255
        imgCrop = img[y - offset:y + h + offset, x - offset:x + w + offset]
 
        imgCropShape = imgCrop.shape
 
        aspectRatio = h / w
 
        if aspectRatio > 1:
            k = imgSize / h
            wCal = math.ceil(k * w)
            imgResize = cv2.resize(imgCrop, (wCal, imgSize))
            imgResizeShape = imgResize.shape
            wGap = math.ceil((imgSize - wCal) / 2)
            imgWhite[:, wGap:wCal + wGap] = imgResize
            imgWhite1=imgWhite.reshape(1,300,300,3)
            prediction= model.predict(imgWhite1)
            index=np.argmax(prediction)
            print(prediction, index)
 
        else:
            k = imgSize / w
            hCal = math.ceil(k * h)
            imgResize = cv2.resize(imgCrop, (imgSize, hCal))
            imgResizeShape = imgResize.shape
            hGap = math.ceil((imgSize - hCal) / 2)
            imgWhite[hGap:hCal + hGap, :] = imgResize
            imgWhite1=imgWhite.reshape(1,300,300,3)
            prediction= model.predict(imgWhite1)
            index=np.argmax(prediction)
            #print(prediction, index)
 
 
        cv2.rectangle(imgOutput, (x - offset, y - offset-50),
                      (x - offset+90, y - offset-50+50), (255, 0, 255), cv2.FILLED)
        cv2.putText(imgOutput, labels[index], (x, y -26), cv2.FONT_HERSHEY_COMPLEX, 1.7, (255, 255, 255), 2)
        cv2.rectangle(imgOutput, (x-offset, y-offset),
                      (x + w+offset, y + h+offset), (255, 0, 255), 4)
 
 
#         cv2.imshow("ImageCrop", imgCrop)
#         cv2.imshow("ImageWhite", imgWhite)
 
    cv2.imshow("Image", imgOutput)
    cv2.waitKey(1)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()

In [ ]:
q